In [1]:
!pip install ortools

In [2]:
import pandas as pd
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [3]:
df=pd.read_excel(open('P4_JobShopData.xlsx', 'rb'),
              sheet_name='Sheet1')

df.head()
df=df.drop('Order', axis=1)

In [19]:
#Convert the data into a list of tuples
result =[tuple(x) for x in df.values.tolist()]
print(result)

[(0, 5, 22), (0, 4, 29), (0, 2, 11), (0, 1, 20), (1, 6, 36), (1, 1, 20), (1, 4, 31), (2, 2, 20), (2, 3, 20), (2, 4, 46), (2, 1, 20), (2, 0, 5), (3, 4, 42), (3, 3, 16), (4, 1, 11), (4, 0, 2), (4, 3, 3), (4, 6, 19), (5, 6, 21), (5, 0, 4), (5, 1, 14), (5, 3, 19), (5, 2, 23), (6, 0, 9), (6, 3, 29), (6, 4, 23), (7, 3, 27), (7, 5, 39), (7, 6, 31), (8, 2, 28), (8, 4, 47), (9, 2, 30), (9, 5, 23), (9, 3, 29), (9, 1, 20), (9, 0, 4), (10, 0, 8), (10, 2, 21), (10, 3, 4), (10, 6, 33), (10, 1, 18), (10, 5, 7), (11, 2, 12), (11, 3, 30), (11, 4, 42), (11, 5, 36), (12, 0, 1), (12, 1, 14), (12, 2, 18), (12, 3, 40), (13, 4, 48), (13, 5, 1), (13, 3, 25), (13, 2, 15), (13, 1, 20), (14, 5, 6), (14, 2, 11), (15, 0, 10), (16, 5, 14), (16, 2, 30), (16, 3, 10), (17, 1, 19), (17, 2, 13), (17, 3, 40), (17, 6, 14), (17, 4, 14), (18, 2, 22), (18, 6, 11), (18, 5, 33), (18, 1, 19), (18, 0, 5), (19, 4, 25), (19, 5, 1), (19, 6, 13)]


In [20]:
#Arrange tuples into a list by Order and then element Order number
r=[[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
x=list(range(0,20))
for j in range(len(x)):
  for i in range(len(df)):
    if result[i][0]==x[j]:
      r[j].append(result[i][1:])

In [21]:
#Preview the resulting list of list of tuples arranged as (m,p)
r

[[(5, 22), (4, 29), (2, 11), (1, 20)],
 [(6, 36), (1, 20), (4, 31)],
 [(2, 20), (3, 20), (4, 46), (1, 20), (0, 5)],
 [(4, 42), (3, 16)],
 [(1, 11), (0, 2), (3, 3), (6, 19)],
 [(6, 21), (0, 4), (1, 14), (3, 19), (2, 23)],
 [(0, 9), (3, 29), (4, 23)],
 [(3, 27), (5, 39), (6, 31)],
 [(2, 28), (4, 47)],
 [(2, 30), (5, 23), (3, 29), (1, 20), (0, 4)],
 [(0, 8), (2, 21), (3, 4), (6, 33), (1, 18), (5, 7)],
 [(2, 12), (3, 30), (4, 42), (5, 36)],
 [(0, 1), (1, 14), (2, 18), (3, 40)],
 [(4, 48), (5, 1), (3, 25), (2, 15), (1, 20)],
 [(5, 6), (2, 11)],
 [(0, 10)],
 [(5, 14), (2, 30), (3, 10)],
 [(1, 19), (2, 13), (3, 40), (6, 14), (4, 14)],
 [(2, 22), (6, 11), (5, 33), (1, 19), (0, 5)],
 [(4, 25), (5, 1), (6, 13)]]

In [22]:
"""Minimal jobshop example."""
import collections
from ortools.sat.python import cp_model


def main():
    """Minimal jobshop problem."""
    # Data.
    jobs_data = r # task = (machine_id, processing_time).
      
    machines_count = 1 + max(task[0] for job in jobs_data for task in job)
    all_machines = range(machines_count)
    # Computes horizon dynamically as the sum of all durations.
    horizon = sum(task[1] for job in jobs_data for task in job)

    # Create the model.
    model = cp_model.CpModel()

    # Named tuple to store information about created variables.
    task_type = collections.namedtuple('task_type', 'start end interval')
    # Named tuple to manipulate solution information.
    assigned_task_type = collections.namedtuple('assigned_task_type',
                                                'start job index duration')

    # Creates job intervals and add to the corresponding machine lists.
    all_tasks = {}
    machine_to_intervals = collections.defaultdict(list)

    for job_id, job in enumerate(jobs_data):
        for task_id, task in enumerate(job):
            machine = task[0]
            duration = task[1]
            suffix = '_%i_%i' % (job_id, task_id)
            start_var = model.NewIntVar(0, horizon, 'start' + suffix)
            end_var = model.NewIntVar(0, horizon, 'end' + suffix)
            interval_var = model.NewIntervalVar(start_var, duration, end_var,
                                                'interval' + suffix)
            all_tasks[job_id, task_id] = task_type(start=start_var,
                                                   end=end_var,
                                                   interval=interval_var)
            machine_to_intervals[machine].append(interval_var)

    # Create and add disjunctive constraints.
    for machine in all_machines:
        model.AddNoOverlap(machine_to_intervals[machine])

    # Precedences inside a job.
    for job_id, job in enumerate(jobs_data):
        for task_id in range(len(job) - 1):
            model.Add(all_tasks[job_id, task_id +
                                1].start >= all_tasks[job_id, task_id].end)

    # Makespan objective.
    obj_var = model.NewIntVar(0, horizon, 'makespan')
    model.AddMaxEquality(obj_var, [
        all_tasks[job_id, len(job) - 1].end
        for job_id, job in enumerate(jobs_data)
    ])
    model.Minimize(obj_var)

    # Creates the solver and solve.
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        print('Solution:')
        # Create one list of assigned tasks per machine.
        assigned_jobs = collections.defaultdict(list)
        for job_id, job in enumerate(jobs_data):
            for task_id, task in enumerate(job):
                machine = task[0]
                assigned_jobs[machine].append(
                    assigned_task_type(start=solver.Value(
                        all_tasks[job_id, task_id].start),
                                       job=job_id,
                                       index=task_id,
                                       duration=task[1]))

        # Create per machine output lines.
        output = ''
        for machine in all_machines:
            # Sort by starting time.
            assigned_jobs[machine].sort()
            sol_line_tasks = 'Machine ' + str(machine) + ': '
            sol_line = '           '

            for assigned_task in assigned_jobs[machine]:
                name = 'job_%i_task_%i' % (assigned_task.job,
                                           assigned_task.index)
                # Add spaces to output to align columns.
                sol_line_tasks += '%-15s' % name

                start = assigned_task.start
                duration = assigned_task.duration
                sol_tmp = '[%i,%i]' % (start, start + duration)
                # Add spaces to output to align columns.
                sol_line += '%-15s' % sol_tmp

            sol_line += '\n'
            sol_line_tasks += '\n'
            output += sol_line_tasks
            output += sol_line

        # Finally print the solution found.
        print(f'Optimal Schedule Length: {solver.ObjectiveValue()}')
        print(output)
    else:
        print('No solution found.')

    # Statistics.
    print('\nStatistics')
    print('  - conflicts: %i' % solver.NumConflicts())
    print('  - branches : %i' % solver.NumBranches())
    print('  - wall time: %f s' % solver.WallTime())


if __name__ == '__main__':
    main()

Solution:
Optimal Schedule Length: 347.0
Machine 0: job_12_task_0  job_10_task_0  job_6_task_0   job_4_task_1   job_15_task_0  job_5_task_1   job_18_task_4  job_2_task_4   job_9_task_4   
           [0,1]          [1,9]          [9,18]         [18,20]        [20,30]        [30,34]        [208,213]      [258,263]      [304,308]      
Machine 1: job_4_task_0   job_12_task_1  job_17_task_0  job_5_task_2   job_1_task_1   job_0_task_3   job_10_task_4  job_18_task_3  job_2_task_3   job_9_task_3   job_13_task_4  
           [0,11]         [11,25]        [25,44]        [44,58]        [58,78]        [151,171]      [171,189]      [189,208]      [238,258]      [284,304]      [327,347]      
Machine 2: job_11_task_0  job_14_task_1  job_2_task_0   job_12_task_2  job_17_task_1  job_10_task_1  job_18_task_0  job_5_task_4   job_0_task_2   job_8_task_0   job_9_task_0   job_16_task_1  job_13_task_3  
           [0,12]         [12,23]        [23,43]        [43,61]        [61,74]        [74,95]        [95